In [ ]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import Tool
import os
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.tools.file_management.write import WriteFileTool
from langchain.vectorstores import FAISS
import requests
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.prebuilt import create_react_agent

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [5]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

In [6]:
data_path = "/Users/lama/Desktop/agent /80 Excel formulas .pdf"

In [ ]:
from langchain_core.documents import Document

# Extract text from a PDF and store it as a Document
text = extract_text_from_pdf(data_path)
documents = [Document(page_content=text)]

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Use a text splitter to break the document into smaller pieces
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(documents)

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# Create embeddings using OpenAI
embeddings = OpenAIEmbeddings()

# Store the embedded chunks in a FAISS vector database
db = FAISS.from_documents(docs , embeddings)

In [ ]:
# Create a retriever from the FAISS vector store
faiss_retriever = db.as_retriever()

In [ ]:
# Initialize GPT-4o-mini 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

In [ ]:
# Build a QA system using the LLM and FAISS retriever
ExcelFormulas= RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=faiss_retriever
)

In [ ]:
def get_weather(city=None):
    # Default to a city if no city is provided
    city = city or "Riyadh"  
    base_url = f"https://wttr.in/{city}?format=%C+%t+%w"  # Add the city to the URL
    response = requests.get(base_url)
    
    if response.status_code == 200:
        return response.text
    else:
        return f"Could not retrieve weather data for {city}."


In [ ]:
# Store chat histories by session ID
_session_histories = {}

# Retrieve or create chat history for a given session
def get_session_history(session_id: str = "default") -> InMemoryChatMessageHistory:
    if session_id not in _session_histories:
        _session_histories[session_id] = InMemoryChatMessageHistory()
    return _session_histories[session_id]

In [ ]:
# Define tools the agent can use: Excel QA and Weather info
tools = [
    Tool(
        name="ExcelFormulas",
        func=ExcelFormulas.run,
        description="useful for when you need to answer questions about Excel formulas, functions, and data manipulation. Input should be a fully formed question.",
    ),
    Tool(
        name="Weather",
        func=get_weather,
        description="Fetches simple weather information without requiring location or API key."
    )
]

In [ ]:
# Initialize the agent with tools and zero-shot reasoning
agent = initialize_agent(
    tools, 
    llm, 
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  
    verbose=True  
)

In [ ]:
# Create an agent with a custom prompt and toolset
def create_agent_with_history(model, tools):
    prompt = "You are a helpful assistant."
    return create_react_agent(model, tools, prompt)


# Interact with the agent and maintain chat history
def ask_agent(agent, question, session_id="default"):
    history = get_session_history(session_id)
    history.add_message(HumanMessage(content=question))

    response = agent.invoke({"messages": history.messages})

    # Look for the last non-empty AI message in the response
    if isinstance(response, dict) and "messages" in response:
        last_ai_message = None
        for msg in response["messages"]:
            if isinstance(msg, AIMessage) and msg.content.strip():
                last_ai_message = msg

        if last_ai_message:
            history.add_message(last_ai_message)

    return response, history

# Ask the agent and print the conversation
def ask_agent_and_print(agent, question, session_id="default"):
    _, history = ask_agent(agent, question, session_id)

    print("🗨️ Conversation:")
    for msg in history.messages:
        if isinstance(msg, HumanMessage):
            print("👤 Human:\n" + msg.content + "\n")
        elif isinstance(msg, AIMessage):
            print("🤖 AI:\n" + msg.content + "\n")


In [191]:
# Clear chat history
def clear_chat_history(session_id="default"):
    # Reset the chat history for the given session
    _session_histories[session_id] = InMemoryChatMessageHistory()
    print(f"Chat history for session '{session_id}' has been cleared.")
    
clear_chat_history("default")


Chat history for session 'default' has been cleared.


In [ ]:
# Create an agent with the given model, tools
agent = create_react_agent(llm, tools, prompt="...")

In [192]:
ask_agent_and_print(agent, "How is the weather today in Riyadh?")

🗨️ Conversation:
👤 Human:
How is the weather today in Riyadh?

🤖 AI:
The weather in Riyadh today is approximately 23°C with patchy rain nearby and a wind speed of 17 km/h.



In [193]:
ask_agent_and_print(agent, "How do I sum a column in Excel?")


🗨️ Conversation:
👤 Human:
How is the weather today in Riyadh?

🤖 AI:
The weather in Riyadh today is approximately 23°C with patchy rain nearby and a wind speed of 17 km/h.

👤 Human:
How do I sum a column in Excel?

🤖 AI:
To sum a column in Excel, you can use the SUM function. For example, if you want to sum the values in column A from row 1 to row 5, you would use the following formula:

```excel
=SUM(A1:A5)
```



In [194]:
ask_agent_and_print(agent, "How can I calculate the average of a range in Excel")

🗨️ Conversation:
👤 Human:
How is the weather today in Riyadh?

🤖 AI:
The weather in Riyadh today is approximately 23°C with patchy rain nearby and a wind speed of 17 km/h.

👤 Human:
How do I sum a column in Excel?

🤖 AI:
To sum a column in Excel, you can use the SUM function. For example, if you want to sum the values in column A from row 1 to row 5, you would use the following formula:

```excel
=SUM(A1:A5)
```

👤 Human:
How can I calculate the average of a range in Excel

🤖 AI:
You can calculate the average of a range in Excel using the AVERAGE function. The syntax is:

```excel
=AVERAGE(A1:A5)
```

Just replace `A1:A5` with the actual range of cells you want to average.



In [195]:
ask_agent_and_print(agent, "How can you count the number of non-empty cells in a column?")


🗨️ Conversation:
👤 Human:
How is the weather today in Riyadh?

🤖 AI:
The weather in Riyadh today is approximately 23°C with patchy rain nearby and a wind speed of 17 km/h.

👤 Human:
How do I sum a column in Excel?

🤖 AI:
To sum a column in Excel, you can use the SUM function. For example, if you want to sum the values in column A from row 1 to row 5, you would use the following formula:

```excel
=SUM(A1:A5)
```

👤 Human:
How can I calculate the average of a range in Excel

🤖 AI:
You can calculate the average of a range in Excel using the AVERAGE function. The syntax is:

```excel
=AVERAGE(A1:A5)
```

Just replace `A1:A5` with the actual range of cells you want to average.

👤 Human:
How can you count the number of non-empty cells in a column?

🤖 AI:
You can count the number of non-empty cells in a column using the COUNTA function in Excel. The syntax is:

```excel
=COUNTA(A1:A5)
```

Replace `A1:A5` with the actual range of cells you want to count. This function will count all non-empt

In [196]:
ask_agent_and_print(agent, "ow to average a column?")


🗨️ Conversation:
👤 Human:
How is the weather today in Riyadh?

🤖 AI:
The weather in Riyadh today is approximately 23°C with patchy rain nearby and a wind speed of 17 km/h.

👤 Human:
How do I sum a column in Excel?

🤖 AI:
To sum a column in Excel, you can use the SUM function. For example, if you want to sum the values in column A from row 1 to row 5, you would use the following formula:

```excel
=SUM(A1:A5)
```

👤 Human:
How can I calculate the average of a range in Excel

🤖 AI:
You can calculate the average of a range in Excel using the AVERAGE function. The syntax is:

```excel
=AVERAGE(A1:A5)
```

Just replace `A1:A5` with the actual range of cells you want to average.

👤 Human:
How can you count the number of non-empty cells in a column?

🤖 AI:
You can count the number of non-empty cells in a column using the COUNTA function in Excel. The syntax is:

```excel
=COUNTA(A1:A5)
```

Replace `A1:A5` with the actual range of cells you want to count. This function will count all non-empt

In [197]:
ask_agent_and_print(agent, "How is the weather today in London")


🗨️ Conversation:
👤 Human:
How is the weather today in Riyadh?

🤖 AI:
The weather in Riyadh today is approximately 23°C with patchy rain nearby and a wind speed of 17 km/h.

👤 Human:
How do I sum a column in Excel?

🤖 AI:
To sum a column in Excel, you can use the SUM function. For example, if you want to sum the values in column A from row 1 to row 5, you would use the following formula:

```excel
=SUM(A1:A5)
```

👤 Human:
How can I calculate the average of a range in Excel

🤖 AI:
You can calculate the average of a range in Excel using the AVERAGE function. The syntax is:

```excel
=AVERAGE(A1:A5)
```

Just replace `A1:A5` with the actual range of cells you want to average.

👤 Human:
How can you count the number of non-empty cells in a column?

🤖 AI:
You can count the number of non-empty cells in a column using the COUNTA function in Excel. The syntax is:

```excel
=COUNTA(A1:A5)
```

Replace `A1:A5` with the actual range of cells you want to count. This function will count all non-empt